In [ ]:
###Important AWS Config Parameters

In [ ]:
aws_profile_name=''
aws_service_name='bedrock-runtime'
aws_region_name=''
opensearch_host_name = "" #pick this value from AWS OpenSearchConsole
aws_embedding_model='amazon.titan-embed-text-v1'
#aws_generation_model='meta.llama2-70b-chat-v1'
aws_generation_model='anthropic.claude-3-sonnet-20240229-v1:0' #Genetative Model
index_name='sgrealestate-index'

### Load All Required Python modules

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, helpers
import boto3
import json

### Initialize AWS OpenSearch client

In [ ]:
service = "aoss"
credentials = boto3.Session(profile_name=aws_profile_name).get_credentials()
auth = AWSV4SignerAuth(credentials, aws_region_name, service)

In [ ]:
client = OpenSearch(
    hosts = [{"host": opensearch_host_name, "port": 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 20
)

### Initialize AWS Bedrock Client

In [ ]:
bedrock = boto3.Session(profile_name=aws_profile_name).client(
 service_name=aws_service_name,
 region_name=aws_region_name)

### Embedding Generation

In [ ]:
def generate_embedding(text):
    print("******Passed text is", text)
    body=json.dumps({"inputText": text})
    data_format='application/json'
    response = bedrock.invoke_model(body=body, modelId=aws_embedding_model, accept=data_format, contentType=data_format)
    response_body = json.loads(response.get('body').read())
    embedding = response_body.get('embedding')
    print("******Embedding response is", embedding)

    return embedding

### Create the prompt

In [ ]:
prompt='which flat was sold for 400000 sgd in Ang Mo Kio. Just answer in one line?'

### Build context

In [ ]:
def perform_knn_search_on_aws_vector_db(embedding_vector):
    document = {
        "size": 15,
        "_source": {"excludes": ["sg_realestate_data_vector"]},
        "query": {
            "knn": {
                 "sg_realestate_data_vector": {
                     "vector": embedding_vector,
                     "k":15
                 }
            }
        }
    }
    response = client.search(
    body = document,
    index = index_name
    )
    return response

In [ ]:
vector=generate_embedding(prompt)
print("******Generated Embedding", vector)

In [ ]:
response=perform_knn_search_on_aws_vector_db(vector)
data=response['hits']['hits']

In [ ]:
data

In [ ]:
context = ''
for item in data:
    context += item['_source']['sg_realestate_raw_data'] + '\n'
print(context)

In [ ]:
augmented_prompt=f'Context - {context}\nBased on the above context, answer this question - {prompt}'

In [ ]:
augmented_prompt

### Call Generation Model - Anthropic Claude3 Sonnet (on AWS Bedrock to generate output)

In [ ]:
response = bedrock.invoke_model(
                modelId=aws_generation_model,
                body=json.dumps(
                    {
                        "anthropic_version": "bedrock-2023-05-31",
                        "max_tokens": 1024,
                        "messages": [
                            {
                                "role": "user",
                                "content": [{"type": "text", "text": augmented_prompt}],
                            }
                        ],
                    }
                ),
            )
# Process and print the response
result = json.loads(response.get("body").read())
input_tokens = result["usage"]["input_tokens"]
output_tokens = result["usage"]["output_tokens"]
output_list = result.get("content", [])

response_text = ""
for output in output_list:
        response_text = response_text + output["text"]
    
print("**** Final Response Received from Model is:  ",response_text)
